In [3]:
# Download CBDB sqlite
!git clone https://github.com/cbdb-project/cbdb_sqlite.git

Cloning into 'cbdb_sqlite'...
Updating files:  52% (9/17)
Updating files:  58% (10/17)
Updating files:  64% (11/17)
Updating files:  70% (12/17)
Updating files:  76% (13/17)
Updating files:  82% (14/17)
Updating files:  88% (15/17)
Updating files:  94% (16/17)
Updating files: 100% (17/17)
Updating files: 100% (17/17), done.


In [4]:
# import glob
import os
latest_file = "./cbdb_sqlite/latest.7z"
print(latest_file)
latest_db = latest_file.split("/")[-1].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

./cbdb_sqlite/latest.7z
latest.db


In [5]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(latest_file).extractall(".")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [6]:
# Create connection to database
import sqlite3
import pandas as pd
import os
global CONN
latest_file = "./cbdb_sqlite/latest.7z"
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [7]:
tables = ['BIOG_MAIN', 'ASSOC_DATA', 'BIOG_ADDR_DATA', 'ALTNAME_DATA', 'KIN_DATA', 'ENTRY_DATA', 'POSTED_TO_OFFICE_DATA', 'STATUS_DATA', 'TEXT_CODES']  
sql_str = ''    
for table in tables:  
    sql_str += f"SELECT '{table}' AS TableName, COUNT(*) AS RowCount FROM {table} UNION ALL "    
# 去除最后一个 UNION ALL  
sql_str = sql_str[:-11]
sql_str 

"SELECT 'BIOG_MAIN' AS TableName, COUNT(*) AS RowCount FROM BIOG_MAIN UNION ALL SELECT 'ASSOC_DATA' AS TableName, COUNT(*) AS RowCount FROM ASSOC_DATA UNION ALL SELECT 'BIOG_ADDR_DATA' AS TableName, COUNT(*) AS RowCount FROM BIOG_ADDR_DATA UNION ALL SELECT 'ALTNAME_DATA' AS TableName, COUNT(*) AS RowCount FROM ALTNAME_DATA UNION ALL SELECT 'KIN_DATA' AS TableName, COUNT(*) AS RowCount FROM KIN_DATA UNION ALL SELECT 'ENTRY_DATA' AS TableName, COUNT(*) AS RowCount FROM ENTRY_DATA UNION ALL SELECT 'POSTED_TO_OFFICE_DATA' AS TableName, COUNT(*) AS RowCount FROM POSTED_TO_OFFICE_DATA UNION ALL SELECT 'STATUS_DATA' AS TableName, COUNT(*) AS RowCount FROM STATUS_DATA UNION ALL SELECT 'TEXT_CODES' AS TableName, COUNT(*) AS RowCount FROM TEXT_CODES"

In [8]:

tablecount_df = pd.read_sql_query(sql_str, CONN).drop_duplicates().dropna()
tablecount_df

,TableName,RowCount
0,BIOG_MAIN,535181
1,ASSOC_DATA,186534
2,BIOG_ADDR_DATA,323736
3,ALTNAME_DATA,180535
4,KIN_DATA,537328
5,ENTRY_DATA,162621
6,POSTED_TO_OFFICE_DATA,393176
7,STATUS_DATA,59786
8,TEXT_CODES,56813


In [9]:
datatypes = ["Number of Persons 人數","Social Associations 社會關係", "Biographical Addresses 地址", "Alternate Names 別名", "Kin Relationships 親屬關係", "Entry into Office 入仕", "Office Postings 任官", "Social Distinction 社會區分", "Texts 著作"]
tablecount_df['TableName'] = tablecount_df['TableName'].replace(dict(zip(tablecount_df['TableName'], datatypes)))  
tablecount_df = tablecount_df.rename(columns={'TableName': "DataTypes"})
tablecount_df 

,DataTypes,RowCount
0,Number of Persons 人數,535181
1,Social Associations 社會關係,186534
2,Biographical Addresses 地址,323736
3,Alternate Names 別名,180535
4,Kin Relationships 親屬關係,537328
5,Entry into Office 入仕,162621
6,Office Postings 任官,393176
7,Social Distinction 社會區分,59786
8,Texts 著作,56813


In [10]:
tablecount_df.to_csv('data_types.csv', index=False)

In [11]:
# Read the content of old JavaScript files
with open('script.js', 'r') as file:  
    js_code = file.read()  
  
# Delete old data  
start_delimiter = 'const chart2Data = ['  
end_delimiter = '];// Chart 2 Data'
start_index = js_code.find(start_delimiter)  
end_index = js_code.find(end_delimiter) + len(end_delimiter)  
js_code = js_code[:start_index] + js_code[end_index:]
# new data
js_data = ',\n'.join(  
    f'{{ label: "{dyname}", value: {count} }}'  
    for dyname,count in zip(tablecount_df["DataTypes"],tablecount_df["RowCount"])  
)
new_data = f'{start_delimiter}\n{js_data}\n{end_delimiter}'  
  
js_code = new_data + "\n" +js_code  
  
# Write updated JavaScript file 
with open('script.js', 'w') as file:  
    file.write(js_code)

In [12]:
# update the date
import datetime  
import re 

svg_path = os.path.join(os.getcwd(), 'images', 'svg4.svg')  
with open(svg_path, 'r') as file:  
    svg_content = file.read()  
  
# get current date  
current_time = datetime.datetime.now().strftime('%Y年%m月%d日') 
current_time_english = datetime.datetime.now().strftime('%B %d, %Y') 
  
updated_svg_content = re.sub(r'更新日期：\d+年\d+月\d+日', f'更新日期：{current_time}', svg_content)    
updated_svg_content = re.sub(r'Updated on: \w+ \d+, \d+', f'Updated on: {current_time_english}', updated_svg_content)
with open(svg_path, 'w') as file:  
    file.write(updated_svg_content) 

In [13]:
# close the db
CONN.close()